In [1]:
import datetime

import ibis
from google.cloud import bigquery
from google.oauth2 import service_account
from sqlalchemy import MetaData, create_engine, func
from sqlalchemy.orm import Session

In [5]:
from napari_dashboard.db_schema.pypi import PyPi
from napari_dashboard.db_update.pypi import (
    build_new_projects_query,
    build_update_query,
)

In [6]:
%load_ext google.cloud.bigquery
%load_ext autoreload

%autoreload 2

In [7]:
!pwd

In [8]:
credentials = service_account.Credentials.from_service_account_file(
    "../key.json"
)
bqclient = bigquery.Client(
    credentials=credentials, project=credentials.project_id
)

In [9]:
engine = create_engine("sqlite:///../sample.db")

metadata = MetaData()
metadata.reflect(bind=engine)
if "pypi_downloads" not in metadata.tables:
    PyPi.metadata.create_all(engine)

In [11]:
QUERRY_STR = f"""
{build_update_query(engine)}

UNION ALL

{build_new_projects_query(engine)}
"""
print(QUERRY_STR)

In [13]:
job = bqclient.query(build_update_query(engine))
with Session(engine) as session:
    before = session.query(func.count(PyPi.id)).scalar()
    elem = []
    for row in job.result():
        elem.append(PyPi(**row))
    session.add_all(elem)
    session.commit()
    after = session.query(func.count(PyPi.id)).scalar()
print("new rows:", after - before)

In [15]:
print(build_update_query(engine))

In [40]:
QUERY = """
SELECT *
FROM `bigquery-public-data.pypi.file_downloads`
WHERE file.project in ('napari', 'partseg', 'ilastik-napari')
  -- Only query the last 30 days of history
  AND DATE(timestamp)
    BETWEEN DATE_SUB(CURRENT_DATE(), INTERVAL 7 DAY)
    AND CURRENT_DATE()
"""
QUERRY2 = """
SELECT
  details.installer.name,
FROM
  `bigquery-public-data.pypi.file_downloads`
WHERE file.project = 'napari'
  -- Only query the last 30 days of history
  AND DATE(timestamp)
    BETWEEN DATE_SUB(CURRENT_DATE(), INTERVAL 5 DAY)
    AND CURRENT_DATE()
GROUP BY details.installer.name
LIMIT 10
"""

In [7]:
df = bqclient.query_and_wait(QUERRY2).to_dataframe()
df

In [11]:
res = bqclient.query_and_wait(QUERY)

In [14]:
res.to_dataframe().to_csv("data.csv")

In [35]:
con = ibis.bigquery.connect(credentials=credentials)

In [36]:
pypi_table = con.table("bigquery-public-data.pypi.file_downloads")

In [37]:
yesterday = ibis.now() - ibis.expr.api.interval(days=1)
data = pypi_table.filter(
    [pypi_table.project == "napari", pypi_table.timestamp > yesterday]
).execute()

In [31]:
sq = ibis.sqlite.connect("../sample.db")

In [32]:
sq.list_tables()

In [40]:
tb = sq.table("pypi_downloads")

In [41]:
tb.select

In [43]:
job = bqclient.query(QUERY)

In [44]:
with Session(engine) as session:
    elem = []
    for row in job.result():
        elem.append(PyPi(**row))
    session.add_all(elem)
    session.commit()

In [13]:
with Session(engine) as session:
    count = session.query(func.count(PyPi.id)).scalar()
count

In [46]:
len(elem)

In [47]:
with Session(engine) as session:
    dist = session.query(PyPi.project).distinct()
list(dist)

In [33]:
import json
from urllib.request import Request, urlopen

url = "https://npe2api.vercel.app/api/extended_summary"
with urlopen(Request(url)) as resp:
    plugins = json.load(resp)

In [74]:
datetime.datetime.now()

In [83]:
list(bqclient.query(build_update_query(engine)))

In [81]:
print(build_update_query(engine))